# Quantum Entanglement and Separability

Date : December 15, 2021

This notebook contains material supporting a paper, currently titled *Five Starter Pieces: Quantum Information Science via Semi-definite Programs*, by Vikesh Siddhu (vsiddhu@protonmail.com) and Sridhar Tayur (stayur@cmu.edu). The paper is available on this **[arXiv](http://arxiv.org/abs/2112.08276)** link. The arXiv paper is released there is under the **[arXiv.org perpetual, non-exclusive license](https://arxiv.org/licenses/nonexclusive-distrib/1.0/license.html)**, and this code is released under the **[MIT license](https://opensource.org/licenses/MIT)**.

This notebook depends upon various packages including [numpy](https://numpy.org/) >= 1.19.5, [picos](https://picos-api.gitlab.io/picos/index.html) >= 2.2.55, and [cvxopt](http://cvxopt.org/) >= 1.2.5.
    
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vsiddhu/SDPQuantumOR/blob/master/Notebook%204%20-%20Quantum%20Entanglement%20and%20Separability.ipynb)

## Introduction


By definition, a quantum state $\rho_{ab}$ on $\mathcal{H}_a \otimes \mathcal{H}_{b}$ is separable if the state can be written as a convex combination of pure states; in other words,
$$
\rho_{ab1} = \sum_i p_i|\psi_i \rangle \langle \psi_i| \otimes |\phi_i \rangle \langle \phi_i|,
$$
where $p_i>0$ and $\sum_i p_i = 1$. A state is entangled if it is not separable. 

A state $\rho_{ab}$ is separable if and only if, for each $k \geq 1$ there is a positive (under) partial transpose (PPT) symmetric extension $\rho_{aB}$, where 
$$\mathcal{H}_B := \bigotimes_{j=1}^k \mathcal{H}_{bj},$$
and all $\mathcal{H}_{bi}$ have same dimension as $\mathcal{H}_b$. For each $k$, the existence of a PPT symmetric extension can be formulated as an SDP. Here we discuss SDPs coming from the cases $k=1$ and $k=2$. 



#### Case 1: SDP for $k=1$

\begin{align}
\begin{aligned}
\text{minimize} \; & \mu \\
\text{subject to} \; & \mu I_{ab} + T_{b1}(\rho_{ab}) \succeq 0
\end{aligned}
\end{align}

If the optimal value of the SDP above is strictly positive, then $\rho_{ab}$ does not have a PPT symmetric extension for $k=1$ and thus $\rho_{ab}$ is entangled.


#### Case 2: SDP for $k=2$

To check for a PPT symmetric extension of $\rho_{ab}$ at $k=2$, we let $\rho_{ab1} = \rho_{ab}$ and solve an SDP

\begin{align}
\begin{aligned}
\text{minimize} \; & \mu \\
\text{subject to} \; & \rho_{ab1} = \rm Tr_{b2}(\rho_{aB}) & \\
& \rho_{aB} = \Pi_{b1b2} \rho_{aB} \Pi_{b1b2}, & \\
& \rm Tr(\rho_{aB}) = 1, & \\
& \rho_{aB} + \mu I_{aB} \succeq 0, &\\
& \mathcal{T}_{b1}(\rho_{aB}) + \mu I_{aB} \succeq 0, & \\
& \mathcal{T}_{b1 b2}(\rho_{aB}) + \mu I_{aB} \succeq 0, & \\
\end{aligned}
\end{align}

where $\Pi_{b1b2} = I_a \otimes S_{b1b2}$, and the swap operator $S_{b1b2}|i \rangle_{b1} |j \rangle_{b2} = |j \rangle_{b1} |i \rangle_{b2}$. If the optimal value of the SDP above is strictly positive, then $\rho_{ab}$ does not have a PPT symmetric extension for $k=2$ and thus $\rho_{ab}$ is entangled.


In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# For Google Colab use, commands installing packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Install PICOS and CVXOPT in Google Colab
if IN_COLAB:
    !pip install -q picos
    !pip install -q cvxopt

In [3]:
import picos as pic
import cvxopt as cvx
#Import functions to help formulate the SDP
from HelperFunctionsforNotebook4 import swapLastTwoOfThreePartyState

In [4]:
print('Solvers supported on this installation of picos:', pic.solvers.all_solvers().keys())


Solvers supported on this installation of picos: dict_keys(['cplex', 'cvxopt', 'ecos', 'glpk', 'gurobi', 'mosek', 'mskfsn', 'scip', 'smcp'])


In [5]:
print('Solvers available to picos on this machine :', pic.solvers.available_solvers())

Solvers available to picos on this machine : ['cvxopt', 'mosek', 'mskfsn']


### Example 1
Given the bell state $| \phi \rangle = \frac{1}{\sqrt{2}}(|00 \rangle + |11 \rangle)$, check whether $\rho_{ab} = | \phi \rangle \langle \phi |$ has a $k=1$ PPT symmetric extension.

In [6]:
#State to check separability
phiMat = np.array([[1.,0.,0.,1.],
                [0.,0.,0.,0.],
                [0.,0.,0.,0.],
                [1.,0.,0.,1.]])/2

da = 2
db1 = 2
dB = db1

In [7]:
#Constants
#----------
pMPic = pic.Constant("phiMat", phiMat)
iMatAB1 = pic.Constant('Iab1', np.eye(da*dB))
shpAB1 = (da*dB,da*dB)

#Variables
#----------
mu = pic.RealVariable("mu")


In [8]:
prob1P = pic.Problem()

#Constraint
#----------
prob1P.add_constraint(pic.partial_transpose(pMPic,subsystems=(1), dimensions=(da,dB)) + mu*iMatAB1 >> 0)


<4×4 LMI Constraint: phiMat.{[2×2]⊗[2×2]ᵀ} + mu·Iab1 ≽ 0>

In [9]:
#Objective
#----------
prob1P.set_objective('min',mu)

#User readable view of the problem being composed in PICOS'
print(prob1P)

---------------------------------------
Semidefinite Program
  minimize mu
  over
    1×1 real variable mu
  subject to
    phiMat.{[2×2]⊗[2×2]ᵀ} + mu·Iab1 ≽ 0
---------------------------------------


In [10]:
#Solve the problem using cvxopt as a solver
prob1P.solve(verbosity=False,solver='cvxopt')

<primal feasible solution pair (claimed optimal) from cvxopt>

In [11]:
print('Status at the end of solving the problem:', prob1P.status)

mu1P =  prob1P.value
print('Least mu obtained from solving the SDP above', mu1P)
print('Since this SDP value is positive, the state is entangled')


Status at the end of solving the problem: optimal
Least mu obtained from solving the SDP above 0.49999999985774324
Since this SDP value is positive, the state is entangled


### Example 2
Check existence of PPT symmetric extension with $k=1$ for a two-qutrit state,
\begin{equation}
\rho_{ab} = \frac{2}{7}|\psi_{+}\rangle \langle\psi_{+}| 
+ \frac{\alpha}{7} \sigma_{+} + \frac{5 - \alpha}{7} S_{ab} \sigma_{+} S_{ab},
\end{equation}
where $|\psi_+ \rangle = \frac{1}{\sqrt{3}}( |00\rangle + |11\rangle + |22 \rangle )$,
$\sigma_{+} = \frac{1}{3} (|01 \rangle \langle 01| + |12 \rangle \langle 12|
+ |20 \rangle \langle 20|)$, $S_{ab}$ is the swap operator, and $0 \leq \alpha \leq 5/2$, 


In [12]:
da = 3
db1 = 3
dB = db1

#Construct input state
idMat = np.eye(da*db1)
psiP = idMat[0] + idMat[4] + idMat[8]
psiP = np.outer(psiP,psiP)
psiP = psiP/np.trace(psiP)

sigPlus = np.outer(idMat[1],idMat[1]) + np.outer(idMat[5],idMat[5]) + np.outer(idMat[6],idMat[6])
sigPlus = sigPlus/np.trace(sigPlus)
sigPlusEx = np.outer(idMat[3],idMat[3]) + np.outer(idMat[7],idMat[7]) + np.outer(idMat[2],idMat[2])
sigPlusEx = sigPlusEx/np.trace(sigPlusEx)

al = 0.75
rhoMat = (2/7)*psiP + (al/7)*sigPlus + ((5-al)/7)*sigPlusEx

In [13]:
#Constants
#----------
pMPic = pic.Constant("rhoAl", rhoMat)
iMatAB = pic.Constant('Iab1', np.eye(da*dB))
shpAB = (da*dB,da*dB)

#Variables
#----------
mu = pic.RealVariable("mu")

In [14]:
prob2P = pic.Problem()

#Constraint
#----------
prob2P.add_constraint(pic.partial_transpose(pMPic,subsystems=(1), dimensions=(da,dB)) + mu*iMatAB >> 0)


<9×9 LMI Constraint: rhoAl.{[3×3]⊗[3×3]ᵀ} + mu·Iab1 ≽ 0>

In [15]:
#Objective
#----------
prob2P.set_objective('min',mu)

#User readable view of the problem being composed in PICOS'
print(prob2P)

--------------------------------------
Semidefinite Program
  minimize mu
  over
    1×1 real variable mu
  subject to
    rhoAl.{[3×3]⊗[3×3]ᵀ} + mu·Iab1 ≽ 0
--------------------------------------


In [16]:
#Solve the problem using cvxopt as a solver
prob2P.solve(verbosity=False,solver='cvxopt')

<primal feasible solution pair (claimed optimal) from cvxopt>

In [17]:
print('Status at the end of solving the problem:', prob2P.status)
mu2P =  prob2P.value


print('The input state paramter alpha is', al)
print('Least mu obtained from solving the SDP above is', mu2P)

Status at the end of solving the problem: optimal
The input state paramter alpha is 0.75
Least mu obtained from solving the SDP above is 0.007501735821370089


The value of the SDP above depends on the parameter $\alpha$. When $\alpha < 1$, $\rho_{ab}$ has no PPT symmetric extension for $k \geq 1$, and hence the SDP value is strictly positive. 


### Example 3
Check for existence of $k=2$ PPT symmetric extension for the state defined in Example 1.

In [18]:
#State to check separability
phiMat = np.array([[1.,0.,0.,1.],
                [0.,0.,0.,0.],
                [0.,0.,0.,0.],
                [1.,0.,0.,1.]])/2

da = 2
db = 2
db1 = db
db2 = db
dB = db1*db2

In [19]:
#Constants
#----------
pMPic = pic.Constant("phiMat", phiMat)
iMatAB = pic.Constant('IaB', np.eye(da*dB))
shpAB = (da*dB,da*dB)
shpAbis = (da,db1,db2)

#Variables
#----------
mu = pic.RealVariable("mu")
ZPic = pic.HermitianVariable("ZaB", shpAB)

In [20]:
prob3P = pic.Problem()

#Constraint
#----------
#Partial Trace returns the original state
prob3P.add_constraint(pic.partial_trace(ZPic, subsystems=(2),dimensions=shpAbis) == pMPic)
#Swapping b1 and b2 spaces leaves the state invariant
prob3P.add_constraint(swapLastTwoOfThreePartyState(ZPic,da,db) == ZPic)

prob3P.add_constraint(pic.trace(ZPic) == 1)
prob3P.add_constraint(ZPic + mu*iMatAB >> 0)

#Partial Transpose constraints for transpose on b1 and b2 spaces
prob3P.add_constraint(pic.partial_transpose(ZPic,subsystems=(1), dimensions=shpAbis)  + mu*iMatAB >> 0)
prob3P.add_constraint(pic.partial_transpose(ZPic,subsystems=(1,2), dimensions=shpAbis) + mu*iMatAB >> 0)





<8×8 Complex LMI Constraint: ZaB.{[2×2]⊗[2×2]ᵀ⊗[2×2]ᵀ} + mu·IaB ≽ 0>

In [21]:
#Objective
#----------
prob3P.set_objective('min',mu)

#User readable view of the problem being composed in PICOS'
print(prob3P)

----------------------------------------------
Complex Semidefinite Program
  minimize mu
  over
    8×8 hermitian variable ZaB
    1×1 real variable mu
  subject to
    ZaB.{[2×2]⊗[2×2]⊗tr([2×2])} = phiMat
    ZaB = shuffled(ZaB,ikjlnm,(2,2,2,2,2,2),C)
    tr(ZaB) = 1
    ZaB + mu·IaB ≽ 0
    ZaB.{[2×2]⊗[2×2]ᵀ⊗[2×2]} + mu·IaB ≽ 0
    ZaB.{[2×2]⊗[2×2]ᵀ⊗[2×2]ᵀ} + mu·IaB ≽ 0
----------------------------------------------


In [22]:
#Solve the problem using cvxopt as a solver
prob3P.solve(verbosity=False,solver='cvxopt')

<primal feasible solution pair (claimed optimal) from cvxopt>

In [23]:
print('Status at the end of solving the problem:', prob3P.status)

mu3P =  prob3P.value

print('Least mu obtained from solving the SDP above is', mu3P)


Status at the end of solving the problem: optimal
Least mu obtained from solving the SDP above is 0.24999999989117092


The optimal value is found to be strictly positive. This is expected from results of Example 1, where no PPT  symmetric extension was found for $k=1$. If there is no symmetric extension for some $k$, then for $k' \geq k$, there is no symmetric extension either. 

### Example 4
Check for existence of $k=2$ PPT symmetric extension for the state $\rho_{ab}$ defined in Example 2

In [24]:
da = 3
db = 3
db1 = db
db2 = db
dB = db1*db2

In [25]:
#Re-construct input state from Example 2
idMat = np.eye(da*db)
psiP = idMat[0] + idMat[4] + idMat[8]
psiP = np.outer(psiP,psiP)
psiP = psiP/np.trace(psiP)

In [26]:
sigPlus = np.outer(idMat[1],idMat[1]) + np.outer(idMat[5],idMat[5]) + np.outer(idMat[6],idMat[6])
sigPlus = sigPlus/np.trace(sigPlus)
sigPlusEx = np.outer(idMat[3],idMat[3]) + np.outer(idMat[7],idMat[7]) + np.outer(idMat[2],idMat[2])
sigPlusEx = sigPlusEx/np.trace(sigPlusEx)

In [27]:
al = 1.95
rhoMat = (2/7)*psiP + (al/7)*sigPlus + ((5-al)/7)*sigPlusEx

In [28]:
#Constants
#----------
pMPic = pic.Constant("phiMat", rhoMat)
iMatAB = pic.Constant('IaB', np.eye(da*dB))
shpAB = (da*dB,da*dB)
shpAbi = (da,db1,db2)

#Variables
#----------
mu = pic.RealVariable("mu")
ZPic = pic.HermitianVariable("ZaB", shpAB)

In [29]:
prob4P = pic.Problem()

#Constraint
#----------
#Partial Trace returns the original state
prob4P.add_constraint(pic.partial_trace(ZPic, subsystems=(2),dimensions=shpAbi) == pMPic)
prob4P.add_constraint(swapLastTwoOfThreePartyState(ZPic,da,db) == ZPic)

prob4P.add_constraint(pic.trace(ZPic) == 1)
prob4P.add_constraint(ZPic + mu*iMatAB >> 0)

# #Partial Transpose constraints for transpose on b1 and b2 spaces
prob4P.add_constraint(pic.partial_transpose(ZPic,subsystems=(1), dimensions=shpAbi)  + mu*iMatAB >> 0)
prob4P.add_constraint(pic.partial_transpose(ZPic,subsystems=(1,2), dimensions=shpAbi) + mu*iMatAB >> 0)

<27×27 Complex LMI Constraint: ZaB.{[3×3]⊗[3×3]ᵀ⊗[3×3]ᵀ} + mu·IaB ≽ 0>

In [30]:
#Objective
#----------
prob4P.set_objective('min',mu)

#User readable view of the problem being composed in PICOS'
print(prob4P)

----------------------------------------------
Complex Semidefinite Program
  minimize mu
  over
    27×27 hermitian variable ZaB
    1×1 real variable mu
  subject to
    ZaB.{[3×3]⊗[3×3]⊗tr([3×3])} = phiMat
    ZaB = shuffled(ZaB,ikjlnm,(3,3,3,3,3,3),C)
    tr(ZaB) = 1
    ZaB + mu·IaB ≽ 0
    ZaB.{[3×3]⊗[3×3]ᵀ⊗[3×3]} + mu·IaB ≽ 0
    ZaB.{[3×3]⊗[3×3]ᵀ⊗[3×3]ᵀ} + mu·IaB ≽ 0
----------------------------------------------


In [31]:
#Solve the problem using cvxopt as a solver
prob4P.solve(verbosity=False,solver='cvxopt')

<primal feasible solution pair (claimed optimal) from cvxopt>

In [32]:
print('Status at the end of solving the problem:', prob4P.status)
mu4P =  prob4P.value

print('The input state paramter alpha is', al)
print('Least mu obtained from solving the SDP above is', mu4P)


Status at the end of solving the problem: optimal
The input state paramter alpha is 1.95
Least mu obtained from solving the SDP above is 0.0003607212345043077


The value of the SDP above depends on the parameter $\alpha$. 

* In Example 2 we found when $\alpha < 1$, $\rho_{\alpha}$ has no PPT symmetric extension for $k = 1$. Here $k=2$ and we also find no symmetric extension. This absense of a PPT symmetric extension is shown here by an optimal $k=2$ SDP value which is strictly positive.

* In Example 2 we also found that $k=1$ PPT symmetric extension exists when $1 \leq \alpha < 2$. However, in the current case we find no PPT symmetric extension for $k=2$; that is, a postive optimal value is returned by the SDP solver. This positive value indicates that the $k=2$ SDP finds entanglement for $1 \leq \alpha <2$, but the $k=1$ SDP does not.

* In Example 2 we found when $2 \leq \alpha \leq 5/2$, there is a symmetric extension for $k=1$. In the current case we find a symmetric extension for $k=2$ as well. This symmetric extension is expected because the state $\rho_{\alpha}$ is separable for $2 \leq \alpha \leq 5/2$.
